In [1]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

if IS_COLAB:
    %pip install -q "ray[tune]" 
    %pip install -U hyperopt

In [2]:
import urllib
import tensorflow as tf
from pathlib import Path
from filelock import FileLock

In [3]:
datadir = Path("./data/raw")
if not datadir.exists():
    datadir.mkdir(parents=True)

In [4]:
DOWNLOAD_URL = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
flowers_dir = tf.keras.utils.get_file(origin=DOWNLOAD_URL,
                                   fname='flower_photos',
                                   cache_dir=datadir,
                                   untar=True)

228827136/228813984 [==============================] - 39s 0us/step


In [5]:
flowers_dir

'data/raw/datasets/flower_photos'

In [6]:
targetsize = (150, 150)

In [7]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  flowers_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=targetsize,
  batch_size=32)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.


2021-12-20 22:03:32.701521: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
X, y = next(iter(train_ds))

In [9]:
datadir

PosixPath('data/raw')

In [10]:
X.shape, y.shape

(TensorShape([32, 150, 150, 3]), TensorShape([32]))

In [11]:
y

<tf.Tensor: shape=(32,), dtype=int32, numpy=
array([2, 1, 4, 3, 1, 2, 1, 2, 4, 1, 4, 4, 3, 4, 1, 2, 0, 4, 1, 1, 1, 4,
       3, 2, 3, 1, 4, 2, 2, 3, 4, 3], dtype=int32)>

In [12]:
def create_model(config):
    import tensorflow as tf
    model = tf.keras.models.Sequential()
    for _ in range(config["conv"]):
        model.add(tf.keras.layers.Conv2D(16, 3, activation="relu"))
        model.add(tf.keras.layers.MaxPool2D((2,2)))
    model.add(tf.keras.layers.Flatten())
    for _ in range(config["hidden"]):
        model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer="adam", 
        loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [13]:
def train_on_flowers(config):
    lock = str(config["datadir"]) + ".lock"
    with FileLock(lock):
        train = tf.keras.utils.image_dataset_from_directory(
                    config["datadir"],
                    validation_split=0.2,
                    subset="training",
                    seed=123,
                    image_size=(150, 150),
                    batch_size=32)
        valid = tf.keras.utils.image_dataset_from_directory(
                    config["datadir"],
                    validation_split=0.2,
                    subset="validation",
                    seed=123,
                    image_size=(150, 150),
                    batch_size=32)

    model = create_model(config)

    model.fit(train, 
    validation_data=valid, 
    verbose=1, epochs=config["epochs"])
    

In [14]:
Path(flowers_dir).absolute()

PosixPath('/Users/rgrouls/Documents/academy/HU/ml-21/5-vision/notebooks/data/raw/datasets/flower_photos')

In [15]:
config = { 
    "conv" : 2,
    "hidden" : 2,
    "epochs" : 2,
    "datadir" : Path(flowers_dir).absolute()
}

In [ ]:
train_on_flowers(config)

In [17]:
import tensorflow.keras as keras
from ray import tune

class TuneReporterCallback(keras.callbacks.Callback):
    """Tune Callback for Keras.
    
    The callback is invoked every epoch.
    """

    def __init__(self, logs={}):
        self.iteration = 0
        super(TuneReporterCallback, self).__init__()

    def on_epoch_end(self, batch, logs={}):
        self.iteration += 1
        tune.report(keras_info=logs, mean_accuracy=logs.get("accuracy"), mean_loss=logs.get("loss"),
         val_loss=logs.get("val_loss"))

In [18]:
def train_on_flowers(config):
    lock = str(config["datadir"]) + ".lock"
    with FileLock(lock):
        train = tf.keras.utils.image_dataset_from_directory(
                    config["datadir"],
                    validation_split=0.2,
                    subset="training",
                    seed=123,
                    image_size=(150, 150),
                    batch_size=32)
        valid = tf.keras.utils.image_dataset_from_directory(
                    config["datadir"],
                    validation_split=0.2,
                    subset="validation",
                    seed=123,
                    image_size=(150, 150),
                    batch_size=32)
        AUTOTUNE = tf.data.AUTOTUNE

        train = train.cache().prefetch(buffer_size=AUTOTUNE)
        valid = valid.cache().prefetch(buffer_size=AUTOTUNE)

    callbacks = [TuneReporterCallback()]
    model = create_model(config)

    model.fit(train, 
    validation_data=valid, 
    callbacks=callbacks,
    verbose=1, epochs=config["epochs"])

In [20]:
config = { 
    "conv" : tune.randint(1, 4),
    "hidden" : tune.randint(1, 4),
    "epochs" : 2,
    "datadir" : Path(flowers_dir).absolute()
}

In [22]:
from ray.tune import JupyterNotebookReporter
from ray.tune.suggest.hyperopt import HyperOptSearch

reporter = JupyterNotebookReporter(overwrite=True)

hyperopt_search = HyperOptSearch(
    metric="val_loss", mode="min")

analysis = tune.run(
    train_on_flowers,
    config=config, 
    search_alg=hyperopt_search,
    progress_reporter=reporter,
    verbose=1, 
    local_dir="logs",
    num_samples=2)

(ImplicitFunc pid=75095) Traceback (most recent call last):
(ImplicitFunc pid=75095)   File "/Users/rgrouls/.pyenv/versions/3.8.12/lib/python3.8/threading.py", line 306, in wait
(ImplicitFunc pid=75095)     gotit = waiter.acquire(True, timeout)
(ImplicitFunc pid=75095) KeyboardInterrupt
(ImplicitFunc pid=75095) 
(ImplicitFunc pid=75095) During handling of the above exception, another exception occurred:
(ImplicitFunc pid=75095) 
(ImplicitFunc pid=75095) Traceback (most recent call last):
(ImplicitFunc pid=75095)   File "/Users/rgrouls/.pyenv/versions/3.8.12/lib/python3.8/queue.py", line 179, in get
(ImplicitFunc pid=75095)     self.not_empty.wait(remaining)
(ImplicitFunc pid=75095)   File "/Users/rgrouls/.pyenv/versions/3.8.12/lib/python3.8/threading.py", line 306, in wait
(ImplicitFunc pid=75095)     gotit = waiter.acquire(True, timeout)
(ImplicitFunc pid=75095)   File "/Users/rgrouls/Library/Caches/pypoetry/virtualenvs/hu-deeplearning-R-urKDRm-py3.8/lib/python3.8/site-packages/ray/wo

86/92 [===========================>..] - ETA: 1s - loss: 3.0351 - accuracy: 0.3507


2021-12-20 22:05:14,146	ERROR tune.py:622 -- Trials did not complete: [train_on_flowers_6d17bfac, train_on_flowers_6fef975e]
2021-12-20 22:05:14,147	INFO tune.py:626 -- Total run time: 42.95 seconds (42.64 seconds for the tuning loop).
2021-12-20 22:05:14,148	WARNING tune.py:630 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


In [87]:
analysis.results_df

/Users/rgrouls/Library/Caches/pypoetry/virtualenvs/hu-deeplearning-R-urKDRm-py3.8/lib/python3.8/site-packages/ray/tune/analysis/experiment_analysis.py:262: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,mean_accuracy,mean_loss,val_loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,neg_mean_loss,experiment_id,...,iterations_since_restore,experiment_tag,keras_info.loss,keras_info.accuracy,keras_info.val_loss,keras_info.val_accuracy,config.conv,config.hidden,config.epochs,config.datadir
trial_id,,,,,,,,,,,,,,,,,,,,,
ccad5380,0.982317,0.056738,0.064487,13.857301,True,None,None,2,-0.056738,d4ef5a2c154a4957a69ac6e1a75ed66a,...,2,"1_conv=1,datadir=_Users_rgrouls_Documents_acad...",0.056738,0.982317,0.064487,0.9787,1,2,2,/Users/rgrouls/Documents/academy/HU/ml-21/5-vi...
cd3038cc,0.982983,0.055655,0.043909,17.212405,True,None,None,2,-0.055655,ff8801a88c7a46209b602e59a26db565,...,2,"2_conv=2,datadir=_Users_rgrouls_Documents_acad...",0.055655,0.982983,0.043909,0.9866,2,2,2,/Users/rgrouls/Documents/academy/HU/ml-21/5-vi...
